In [2]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, ReLU, GlobalAveragePooling1D, Dense, Dropout, Flatten, Add,  Input, MaxPooling1D, concatenate, LSTM 
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import Precision, Recall
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:

def load_ts_file(file_path):
    """
    Función para cargar archivos .ts y separar características y etiquetas.
    Asume que las características están separadas por comas y las etiquetas están separadas por dos puntos (:).
    """
    features = []
    labels = []

    # Leer el archivo línea por línea
    with open(file_path, 'r') as file:
        for line in file:
            # Separar características y etiquetas usando ":"
            line_split = line.strip().split(':')
            # Verificar si la línea tiene características y una etiqueta
            if len(line_split) == 2:
                # Las características están antes del ":"
                features.append([float(val) for val in line_split[0].split(',')])
                # La etiqueta está después del ":" (solo una columna, un valor)
                label = int(line_split[1].strip())  # Se elimina cualquier espacio adicional
                labels.append(label)
    # Convertir a DataFrame para las características y convertir las etiquetas en array
    X = pd.DataFrame(features)
    y = pd.DataFrame(labels, columns=['Etiqueta'])

    return X, y

# Ruta del archivo
file_path = 'I:/Maestria/IA/Proyecto/Proyecto/Dataset/ElectricDeviceDetection/ElectricDeviceDetection_TRAIN.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FaultDetectionA/FaultDetectionA_TRAIN.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FaultDetectionB/FaultDetectionB_TRAIN.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FordA/FordA_TRAIN.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FordB/FordB_TRAIN.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FreezerRegularTrain/FreezerRegularTrain_TRAIN.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FreezerSmallTrain/FreezerSmallTrain_TRAIN.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/KeplerLightCurves/KeplerLightCurves_TRAIN.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/StarLightCurves/StarLightCurves_TRAIN.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/Wafer/Wafer_TRAIN.ts'

# Cargar los datos
X_train, y_train = load_ts_file(file_path)

file_path = 'I:/Maestria/IA/Proyecto/Proyecto/Dataset/ElectricDeviceDetection/ElectricDeviceDetection_TEST.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FaultDetectionA/FaultDetectionA_TEST.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FaultDetectionB/FaultDetectionB_TEST.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FordA/FordA_TEST.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FordB/FordB_TEST.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FreezerRegularTrain/FreezerRegularTrain_TEST.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FreezerSmallTrain/FreezerSmallTrain_TEST.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/KeplerLightCurves/KeplerLightCurves_TEST.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/StarLightCurves/StarLightCurves_TEST.ts'
#'I:/Maestria/IA/Proyecto/Proyecto/Dataset/Wafer/Wafer_TEST.ts'

# Cargar los datos
X_test, y_test = load_ts_file(file_path)

y_train = pd.get_dummies(y_train, columns=['Etiqueta'])
y_test = pd.get_dummies(y_test, columns=['Etiqueta'])
# Mostrar algunas filas de los datos cargados
print("Características (X):\n", X_train.head())
print("Etiquetas (y):\n", y_train)

X_train, y_train = shuffle(X_train, y_train)

X_train = np.expand_dims(X_train, axis=1)  # Agregar una dimensión para pasos temporales si no existen
X_test = np.expand_dims(X_test, axis=1)    # Lo mismo para el conjunto de prueba

y_train = pd.DataFrame(y_train).reset_index(drop=True)


Características (X):
     0     1     2     3     4     5     6     7     8     9    ...   278  \
0  12.0  18.0  11.0  11.0  19.0  17.0   4.0   6.0   8.0  12.0  ...  10.0   
1  12.0   9.0  11.0   7.0  12.0  14.0  10.0  10.0  10.0   6.0  ...   4.0   
2   8.0   5.0  10.0  11.0   9.0  10.0   7.0  18.0  11.0   8.0  ...  13.0   
3  16.0  15.0  13.0  14.0   8.0  16.0  14.0   9.0   9.0   8.0  ...  12.0   
4   8.0   6.0   5.0  11.0   7.0   4.0  10.0   8.0   8.0   4.0  ...  19.0   

    279   280   281   282  283   284   285   286   287  
0   9.0  11.0   8.0   4.0  7.0   3.0   6.0   3.0   6.0  
1   7.0   9.0  11.0   6.0  9.0   7.0  10.0   4.0   9.0  
2   7.0   6.0   7.0   1.0  2.0   6.0   7.0   8.0   6.0  
3  11.0   6.0   3.0  11.0  9.0   2.0   6.0   6.0   3.0  
4  13.0  15.0  17.0  10.0  8.0  12.0  16.0  15.0  13.0  

[5 rows x 288 columns]
Etiquetas (y):
     Etiqueta_1  Etiqueta_2  Etiqueta_3  Etiqueta_4  Etiqueta_5  Etiqueta_6  \
0         True       False       False       False       Fals

## Hiperparametros generales

In [3]:

epochs = 400
batch_size = 32


## MLP

In [5]:
# Definir el número de neuronas en función de la longitud de la serie temporal y el número de capas ocultas
def create_mlp(input_shape, num_classes, num_layers=3):
    model = Sequential()
    model.add(Dense(units=input_shape[1], input_shape=input_shape, activation='relu'))
    for i in range(num_layers):
        neurons = input_shape[1] * (num_layers - i)
        model.add(Dense(units=neurons, activation='relu'))
        model.add(Dropout(0.3 / (2 ** i)))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [ ]:
# Ajustar el input_shape correctamente
input_shape = (X_train.shape[1], X_train.shape[2])  # Usar solo dos dimensiones
print(input_shape[1])
num_classes = y_train.shape[1]
model = create_mlp(input_shape, num_classes, num_layers=3)
model.summary()

# Compilar el modelo
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# Callback para reducir el learning rate en plateau
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=epochs, validation_split=0.2, verbose=2,
                    callbacks=[early_stopping, reduce_lr])

# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

## FCN

In [6]:
# Función para crear el modelo FCN
def create_fcn_model(input_shape, num_classes):
    model = Sequential()

    # Primera capa convolucional
    model.add(Conv1D(filters=128, kernel_size=8, padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(ReLU())

    # Segunda capa convolucional
    model.add(Conv1D(filters=256, kernel_size=5, padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())

    # Tercera capa convolucional
    model.add(Conv1D(filters=128, kernel_size=3, padding='same'))
    model.add(BatchNormalization())
    model.add(ReLU())

    # Capa de Global Average Pooling
    model.add(GlobalAveragePooling1D())

    # Capa de salida (softmax)
    model.add(Dense(num_classes, activation='softmax'))

    return model




In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2])  # Dimensiones de entrada de la serie temporal
num_classes = y_train.shape[1]  # Número de clases (asumiendo one-hot encoding)
model = create_fcn_model(input_shape, num_classes)
model.summary()

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
loss='categorical_crossentropy', metrics=['accuracy'])

# Reducir la tasa de aprendizaje cuando el progreso sea lento
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=2, callbacks=[early_stopping, reduce_lr])

# Mostrar el resumen de la arquitectura del modelo


# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

## ResNet

In [7]:

def residual_block(x, filters, kernel_size=3):
    # Primera convolución
    conv1 = Conv1D(filters=filters, kernel_size=kernel_size, padding='same', kernel_regularizer=l2(0.01))(x)
    conv1 = BatchNormalization()(conv1)
    conv1 = ReLU()(conv1)

    # Segunda convolución
    conv2 = Conv1D(filters=filters, kernel_size=kernel_size, padding='same', kernel_regularizer=l2(0.01))(conv1)
    conv2 = BatchNormalization()(conv2)
    conv2 = ReLU()(conv2)

    # Tercera convolución
    conv3 = Conv1D(filters=filters, kernel_size=kernel_size, padding='same', kernel_regularizer=l2(0.01))(conv2)
    conv3 = BatchNormalization()(conv3)

    # Proyección para hacer coincidir las dimensiones de entrada y salida
    shortcut = Conv1D(filters=filters, kernel_size=1, padding='same', kernel_regularizer=l2(0.01))(x)
    shortcut = BatchNormalization()(shortcut)

    # Agregar la conexión residual (entrada proyectada)
    output = Add()([shortcut, conv3])
    output = ReLU()(output)

    return output

# Función para crear el modelo ResNet
def create_resnet_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Primer bloque residual
    x = residual_block(inputs, filters=64)

    # Segundo bloque residual
    x = residual_block(x, filters=128)

    # Tercer bloque residual
    x = residual_block(x, filters=128)

    # Capa de Global Average Pooling
    x = GlobalAveragePooling1D()(x)

    # Capa de salida (softmax)
    outputs = Dense(num_classes, activation='softmax')(x)

    # Definir el modelo
    model = Model(inputs=inputs, outputs=outputs)

    # Compilar el modelo
    

    return model




In [ ]:
# Crear el modelo ResNet
input_shape = (X_train.shape[1], X_train.shape[2])  # Dimensión de entrada
num_classes = y_train.shape[1]  # Número de clases (asumiendo que y_train es one-hot encoded)
model = create_resnet_model(input_shape, num_classes)

model.summary()

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=2, callbacks=[early_stopping, reduce_lr])

# Mostrar el resumen de la arquitectura del modelo


# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

## IncepctionFCN

In [8]:
# Función para crear un bloque de Inception
def inception_block(input_tensor, filters):
    # Convolución 1x1
    conv1 = Conv1D(filters=filters, kernel_size=1, padding='same', activation=None)(input_tensor)
    conv1 = BatchNormalization()(conv1)
    conv1 = ReLU()(conv1)

    # Convolución 10x1
    conv10 = Conv1D(filters=filters, kernel_size=10, padding='same', activation=None)(conv1)
    conv10 = BatchNormalization()(conv10)
    conv10 = ReLU()(conv10)

    # Convolución 20x1
    conv20 = Conv1D(filters=filters, kernel_size=20, padding='same', activation=None)(conv10)
    conv20 = BatchNormalization()(conv20)
    conv20 = ReLU()(conv20)

    # MaxPooling seguido de convolución 1x1
    pool = MaxPooling1D(pool_size=3, strides=1, padding='same')(input_tensor)
    pool_conv = Conv1D(filters=filters, kernel_size=1, padding='same', activation=None)(pool)
    pool_conv = BatchNormalization()(pool_conv)
    pool_conv = ReLU()(pool_conv)

    # Concatenación de las salidas
    output = concatenate([conv1, conv10, conv20, pool_conv], axis=-1)
    
    return output

# FCN block basado en las imágenes proporcionadas
def fcn_block(input_tensor):
    # Seis capas de convolución, cada una con tamaño de filtro 128 y diferentes tamaños de kernel
    x = Conv1D(filters=128, kernel_size=1, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv1D(filters=128, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv1D(filters=128, kernel_size=5, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv1D(filters=128, kernel_size=5, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv1D(filters=128, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv1D(filters=128, kernel_size=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

# Creación del modelo InceptionFCN
def create_inceptionfcn_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    
    # Módulo Inception (dos bloques como se indica en la descripción)
    x = inception_block(inputs, filters=64)
    x = inception_block(x, filters=64)

    # Módulo FCN (Shortcut module)
    fcn_output = fcn_block(inputs)

    # Concatenar la salida del módulo Inception y el FCN (shortcut module)
    combined = concatenate([x, fcn_output])

    # Global Average Pooling seguido de Dropout
    combined = GlobalAveragePooling1D()(combined)
    combined = Dropout(0.5)(combined)
    
    # Capa de salida softmax
    outputs = Dense(num_classes, activation='softmax')(combined)
    
    # Definir el modelo
    model = Model(inputs=inputs, outputs=outputs)
    

    
    return model

# Mostrar resumen del modelo


In [ ]:
model.summary()
input_shape = (X_train.shape[1], X_train.shape[2])   # Ajustar a la dimensión de tus datos
num_classes = y_train.shape[1]      # Ajustar al número de clases en tu dataset

model = create_inceptionfcn_model(input_shape, num_classes)

# Compilar el modelo con Adam y una tasa de aprendizaje de 1e-3
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks para Early Stopping y reducción de la tasa de aprendizaje
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)

# Entrenamiento (X_train y y_train deben estar preparados previamente)
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=2, callbacks=[early_stopping, reduce_lr])

# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

## S2swa

In [9]:
# Definir el Encoder-Decoder LSTM

def encoder_decoder_lstm(input_shape, n_units=15, target_length=15, dropout_rate=0.3, l2_reg=1e-3):
    inputs = Input(shape=input_shape)
    x = inputs
    # Tres capas LSTM con Batch Normalization, Regularización L2 y Dropout (Encoder)
    for _ in range(2):
        x = LSTM(n_units, return_sequences=True, kernel_regularizer=l2(l2_reg))(x)
        x = BatchNormalization()(x)
        x = Dropout(dropout_rate)(x)
    encoder_output, state_h, state_c = LSTM(n_units, return_sequences=True, return_state=True, kernel_regularizer=l2(l2_reg))(x)
    encoder_states = [state_h, state_c]
    
    # Longitud de salida fija L (Decoder)
    decoder_lstm_layer = LSTM(n_units, return_sequences=True, return_state=True, kernel_regularizer=l2(l2_reg))
    decoder_outputs, _, _ = decoder_lstm_layer(encoder_output, initial_state=encoder_states)
    # Dos capas LSTM con Batch Normalization y Dropout
    for _ in range(1):
        decoder_outputs = LSTM(n_units, return_sequences=True, kernel_regularizer=l2(l2_reg))(decoder_outputs)
        decoder_outputs = BatchNormalization()(decoder_outputs)
        decoder_outputs = Dropout(dropout_rate)(decoder_outputs)
    
    return inputs, decoder_outputs

# Definir el modelo S2SwA
def create_s2swa_model(input_shape, target_length, n_units=15, fc_units=400, num_classes=10, dropout_rate=0.3, l2_reg=1e-3):
    # Encoder-Decoder
    inputs, decoder_outputs = encoder_decoder_lstm(input_shape, n_units, target_length, dropout_rate, l2_reg)
    
    # Capa totalmente conectada
    fc = Dense(fc_units, activation='relu', kernel_regularizer=l2(l2_reg))(decoder_outputs)
    fc = Dropout(0.5)(fc)
    
    # Aplanar antes de la capa de salida
    fc = Flatten()(fc)
    
    # Capa de salida
    outputs = Dense(num_classes, activation='softmax')(fc)
    
    # Definir el modelo completo
    model = Model(inputs, outputs)
    
    # Compilar el modelo con SGD y momentum
    
    
    return model

# Mostrar el resumen del modelo




In [ ]:

# Crear el modelo
input_shape = (X_train.shape[1], X_train.shape[2])  # Ajustar al tamaño de entrada de tus datos
target_length = 15     # Longitud fija para la secuencia de salida (L)
num_classes = y_train.shape[1]       # Número de clases en el dataset
model = create_s2swa_model(input_shape, target_length, n_units=15, num_classes=num_classes, dropout_rate=0.3, l2_reg=1e-3)
model.summary()

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
# Callbacks para Early Stopping y reducción de la tasa de aprendizaje
# Callbacks para Early Stopping y reducción de la tasa de aprendizaje
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)

# Entrenamiento del modelo
# X_train y y_train del mismo conjunto usado en los modelos anteriores
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=2, callbacks=[early_stopping, reduce_lr])

# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

## CONV-LSTM

In [10]:
class Model_CONV_LSTM(Model):
    def __init__(self, num_n_deep=[64, 64, 512, 264, 6], drop_out=[0.3, 0.3], **kwargs):
        super(Pasos_Desbaste, self).__init__()

        self.num_classes = num_n_deep[-1]  # Asumimos que la última posición de num_n_deep indica el número de clases
        self.num_n_deep = num_n_deep
        self.drop_out = drop_out

        # Inicialización de capas en el constructor
        # Capas Conv1D con diferentes filtros
        self.Pred_Layer1_conv1 = layers.Conv1D(filters=64, kernel_size=32, padding='same', activation='relu')
        self.Pred_Layer1_BatchNorm = layers.BatchNormalization()  # BatchNormalization después de la capa Conv1D
        self.Pred_Layer2_conv2 = layers.Conv1D(filters=32, kernel_size=16, padding='same', activation='relu')
        self.Pred_Layer2_BatchNorm = layers.BatchNormalization()
        self.Pred_Layer3_conv3 = layers.Conv1D(filters=16, kernel_size=8, padding='same', activation='relu')
        self.Pred_Layer3_BatchNorm = layers.BatchNormalization()
        self.Pred_Layer4_maxpool = layers.MaxPooling1D(pool_size=2)

        # Aumentar el número de capas LSTM
        self.Pred_Layer5_LSTM = layers.LSTM(units=self.num_n_deep[1], return_sequences=True, dropout=0.3, kernel_regularizer=tf.keras.regularizers.L2(0.007))
        self.Pred_Layer5_LayerNorm = layers.LayerNormalization()  # LayerNormalization después de LSTM
        self.Pred_Layer6_LSTM = layers.LSTM(units=self.num_n_deep[1], return_sequences=True, dropout=0.3, kernel_regularizer=tf.keras.regularizers.L2(0.007))
        self.Pred_Layer6_LayerNorm = layers.LayerNormalization()  # LayerNormalization después de LSTM
        self.Pred_Layer7_LSTM = layers.LSTM(units=self.num_n_deep[1], return_sequences=True, dropout=0.4, kernel_regularizer=tf.keras.regularizers.L2(0.007))
        self.Pred_Layer7_LayerNorm = layers.LayerNormalization()  # LayerNormalization después de LSTM
        self.Pred_Layer8_LSTM = layers.LSTM(units=self.num_n_deep[1], return_sequences=True, dropout=0.4, kernel_regularizer=tf.keras.regularizers.L2(0.007))
        self.Pred_Layer8_LayerNorm = layers.LayerNormalization()  # LayerNormalization después de LSTM

        # Reducir el número de capas de Atención
        self.Attention_Layer1 = layers.Attention()  # Primera capa de atención

        self.Pred_GlobalPool = layers.GlobalAveragePooling1D()
        self.Pred_Layer10_Dense = layers.Dense(units=self.num_n_deep[2], activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.007))
        self.Pred_Layer10_BatchNorm = layers.BatchNormalization()  # BatchNormalization después de Dense
        self.Pred_Layer11_Dropout = layers.Dropout(rate=self.drop_out[0])
        self.Pred_Layer12_Dense = layers.Dense(units=self.num_n_deep[3], activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.007))
        self.Pred_Layer12_BatchNorm = layers.BatchNormalization()  # BatchNormalization después de Dense
        self.Pred_Layer13_Dropout = layers.Dropout(rate=self.drop_out[1])
        self.Pred_Layer14_Dense = layers.Dense(units=self.num_classes, activation='softmax')

    def call(self, inputs, out_before=0, pas_bef=False):
        # Capas Conv1D con diferentes filtros
        x1 = self.Pred_Layer1_conv1(inputs)
        x1 = self.Pred_Layer1_BatchNorm(x1)
        x2 = self.Pred_Layer2_conv2(inputs)
        x2 = self.Pred_Layer2_BatchNorm(x2)
        x3 = self.Pred_Layer3_conv3(inputs)
        x3 = self.Pred_Layer3_BatchNorm(x3)

        # Concatenar salidas de capas Conv1D
        x = layers.Concatenate()([x1, x2, x3])

        # Capas LSTM
        x_lstm1 = self.Pred_Layer5_LSTM(x)
        x_lstm1 = self.Pred_Layer5_LayerNorm(x_lstm1)
        x_lstm2 = self.Pred_Layer6_LSTM(x_lstm1)
        x_lstm2 = self.Pred_Layer6_LayerNorm(x_lstm2)
        x_lstm3 = self.Pred_Layer7_LSTM(x_lstm2)
        x_lstm3 = self.Pred_Layer7_LayerNorm(x_lstm3)
        x_lstm4 = self.Pred_Layer8_LSTM(x_lstm3)
        x_lstm4 = self.Pred_Layer8_LayerNorm(x_lstm4)

        # Concatenar salidas de capas LSTM
        x = layers.Concatenate()([x_lstm1, x_lstm2, x_lstm3, x_lstm4])

        # Aplicar atención
        x = self.Attention_Layer1([x, x])

        # GlobalAveragePooling1D antes de las capas densas
        x = self.Pred_GlobalPool(x)

        # Capas densas
        x = self.Pred_Layer10_Dense(x)
        x = self.Pred_Layer10_BatchNorm(x)
        x = self.Pred_Layer11_Dropout(x)
        x = self.Pred_Layer12_Dense(x)
        x = self.Pred_Layer12_BatchNorm(x)
        x = self.Pred_Layer13_Dropout(x)
        if pas_bef:
            x = layers.Concatenate()([x, out_before])
        x = self.Pred_Layer14_Dense(x)
        return x




In [ ]:
# Crear y compilar el modelo
input_shape = (X_train.shape[1], X_train.shape[2])  # Ajustar al tamaño de entrada de tus datos
num_classes = y_train.shape[1]  # Número de clases en los datos de entrenamiento
model = Model_CONV_LSTM(num_n_deep=[64, 64, 512, 264, num_classes])
model.build((None, *input_shape))
# Mostrar el resumen del modelo
model.summary()
# Compilar el modelo con Adam
model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks para Early Stopping y reducción de la tasa de aprendizaje
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)

# Callback para el planificador de tasa de aprendizaje
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
learning_rate_scheduler = LearningRateScheduler(scheduler)

# Entrenamiento del modelo
history = model.fit(X_train, y_train,
                    epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=2,
                    callbacks=[early_stopping, reduce_lr, learning_rate_scheduler])

# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
def load_ts_file(file_path):
    """
    Función para cargar archivos .ts y separar características y etiquetas.
    Asume que las características están separadas por comas y las etiquetas están separadas por dos puntos (:).
    """
    features = []
    labels = []

    # Leer el archivo línea por línea
    with open(file_path, 'r') as file:
        for line in file:
            # Separar características y etiquetas usando ":"
            line_split = line.strip().split(':')
            # Verificar si la línea tiene características y una etiqueta
            if len(line_split) == 2:
                # Las características están antes del ":"
                features.append([float(val) for val in line_split[0].split(',')])
                # La etiqueta está después del ":" (solo una columna, un valor)
                label = int(line_split[1].strip())  # Se elimina cualquier espacio adicional
                labels.append(label)
    # Convertir a DataFrame para las características y convertir las etiquetas en array
    X = pd.DataFrame(features)
    y = pd.DataFrame(labels, columns=['Etiqueta'])

    return X, y
# Callbacks comunes
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
learning_rate_scheduler = LearningRateScheduler(scheduler)

# Definir los modelos disponibles
model_functions = {
    'MLP': create_mlp,
    'FCN': create_fcn_model,
    'ResNet': create_resnet_model,
    'InceptionFCN': create_inceptionfcn_model,
    'Seq2SeqwA': create_s2swa_model,
    'CONV_LSTM': lambda input_shape, num_classes: Model_CONV_LSTM(num_n_deep=[64, 64, 512, 264, num_classes])
}

# Definir los datasets disponibles
file_paths_train = [
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/ElectricDeviceDetection/ElectricDeviceDetection_TRAIN.ts',
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FaultDetectionA/FaultDetectionA_TRAIN.ts',
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FaultDetectionB/FaultDetectionB_TRAIN.ts'
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FordA/FordA_TRAIN.ts'
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FordB/FordB_TRAIN.ts'
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FreezerRegularTrain/FreezerRegularTrain_TRAIN.ts'
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FreezerSmallTrain/FreezerSmallTrain_TRAIN.ts'
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/KeplerLightCurves/KeplerLightCurves_TRAIN.ts'
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/StarLightCurves/StarLightCurves_TRAIN.ts'
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/Wafer/Wafer_TRAIN.ts'
]

file_paths_test = [
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/ElectricDeviceDetection/ElectricDeviceDetection_TEST.ts',
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FaultDetectionA/FaultDetectionA_TEST.ts',
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FaultDetectionB/FaultDetectionB_TEST.ts'
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FordA/FordA_TEST.ts'
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FordB/FordB_TEST.ts'
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FreezerRegularTrain/FreezerRegularTrain_TEST.ts'
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/FreezerSmallTrain/FreezerSmallTrain_TEST.ts'
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/KeplerLightCurves/KeplerLightCurves_TEST.ts'
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/StarLightCurves/StarLightCurves_TEST.ts'
    'I:/Maestria/IA/Proyecto/Proyecto/Dataset/Wafer/Wafer_TEST.ts'

]

# Inicializar DataFrame para almacenar resultados
results_df = pd.DataFrame(columns=['Model', 'Dataset', 'Accuracy', 'Precision', 'Recall', 'Training Time'])

# Bucle a través de cada dataset y cada modelo
for train_path, test_path in zip(file_paths_train, file_paths_test):
    # Cargar los datos
    X_train, y_train = load_ts_file(train_path)
    X_test, y_test = load_ts_file(test_path)

    # Preprocesamiento de datos
    y_train = pd.get_dummies(y_train, columns=['Etiqueta'])
    y_test = pd.get_dummies(y_test, columns=['Etiqueta'])
    X_train, y_train = shuffle(X_train, y_train)
    X_train = np.expand_dims(X_train, axis=1)
    X_test = np.expand_dims(X_test, axis=1)

    input_shape = (X_train.shape[1], X_train.shape[2])
    num_classes = y_train.shape[1]

    for model_name, model_function in model_functions.items():
        # Crear el modelo
        if model_name == 'CONV_LSTM':
            model = model_function(num_n_deep=[64, 64, 512, 264, num_classes])
            model.build((None, *input_shape))
        if model_name == "S2SwA":
            model = model_fn(input_shape, target_length = 15, n_units=15, num_classes=num_classes, dropout_rate=0.3, l2_reg=1e-3)

        else:
            model = model_function(input_shape, num_classes)

        # Compilar el modelo
        model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])

        # Entrenar el modelo
        start_time = time.time()
        history = model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[early_stopping, reduce_lr, learning_rate_scheduler],
            verbose=0
        )
        training_time = time.time() - start_time

        # Evaluar el modelo
        scores = model.evaluate(X_test, y_test, verbose=0)
        accuracy = scores[1]
        precision = scores[2]
        recall = scores[3]

        # Almacenar resultados
        results_df = pd.concat([results_df, pd.DataFrame([{
            'Model': model_name,
            'Dataset': train_path.split('/')[-2],  # Extraer nombre del dataset
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'Training Time': training_time
        }])], ignore_index=True)

# Mostrar los resultados
print(results_df)


C:\Users\cearr\AppData\Local\Temp\ipykernel_5904\1296174688.py:126: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([{


In [10]:
# Definir una lista de modelos a evaluar
models_to_evaluate = [
    ("MLP", create_mlp),
    ("FCN", create_fcn_model),
    ("ResNet", create_resnet_model),
    ("InceptionFCN", create_inceptionfcn_model),
    ("S2SwA", create_s2swa_model),
    ("CONV_LSTM", lambda input_shape, num_classes: Model_CONV_LSTM(num_n_deep=[64, 64, 512, 264, num_classes]))
]

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
learning_rate_scheduler = LearningRateScheduler(scheduler)

# Inicializar el dataframe para almacenar los resultados
results_df = pd.DataFrame(columns=["Model", "Accuracy", "Training Time (s)", "Validation Loss"])

# Callbacks comunes
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)

# Evaluar cada modelo
def evaluate_models(models, X_train, y_train, X_test, y_test, epochs, batch_size):
    results = []
    input_shape = (X_train.shape[1], X_train.shape[2])
    num_classes = y_train.shape[1]

    for model_name, model_fn in models:
        print(f"Evaluando modelo: {model_name}")
        try:
            # Crear el modelo
            if model_name == "CONV_LSTM":
                model = model_fn(input_shape, num_classes)
                model.build((None, *input_shape))
            if model_name == "S2SwA":
                model = model_fn(input_shape, target_length, n_units=15, num_classes=num_classes, dropout_rate=0.3, l2_reg=1e-3)
            else:
                model = model_fn(input_shape, num_classes)

            # Compilar el modelo
            model.compile(Adam(learning_rate=0.001),
                          loss='categorical_crossentropy', metrics=['accuracy'])

            # Medir el tiempo de entrenamiento
            start_time = time.time()
            history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=2, callbacks=[early_stopping, reduce_lr, learning_rate_scheduler])
            end_time = time.time()

            # Evaluar el modelo en el conjunto de prueba
            test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
            training_time = end_time - start_time

            # Almacenar los resultados
            results.append({
                "Model": model_name,
                "Accuracy": test_acc,
                "Training Time (s)": training_time,
                "Validation Loss": min(history.history['val_loss'])
            })
        except Exception as e:
            print(f"Error evaluando el modelo {model_name}: {e}")
            results.append({
                "Model": model_name,
                "Accuracy": None,
                "Training Time (s)": None,
                "Validation Loss": None
            })
    return pd.DataFrame(results)

# Ejecutar la evaluación de los modelos
results_df = evaluate_models(models_to_evaluate, X_train, y_train, X_test, y_test, epochs=epochs, batch_size=batch_size)

# Ordenar por la mejor precisión
results_df = results_df.sort_values(by=["Accuracy", "Training Time (s)"], ascending=[False, True])

# Mostrar los resultados
print("\nResultados comparativos de los modelos:\n")
print(results_df)

# Resaltar el mejor modelo
best_model = results_df.iloc[0]
print(f"\nEl mejor modelo es: {best_model['Model']} con una precisión de {best_model['Accuracy']:.4f} y un tiempo de entrenamiento de {best_model['Training Time (s)']:.2f} segundos.")

Evaluando modelo: MLP
Epoch 1/400
2/2 - 2s - loss: 16.0879 - accuracy: 0.1887 - val_loss: 32.4382 - val_accuracy: 0.0714 - lr: 0.0010 - 2s/epoch - 985ms/step
Epoch 2/400
2/2 - 0s - loss: 15.8921 - accuracy: 0.3019 - val_loss: 12.4943 - val_accuracy: 0.2143 - lr: 0.0010 - 44ms/epoch - 22ms/step
Epoch 3/400
2/2 - 0s - loss: 9.9697 - accuracy: 0.2264 - val_loss: 4.5003 - val_accuracy: 0.0714 - lr: 0.0010 - 42ms/epoch - 21ms/step
Epoch 4/400
2/2 - 0s - loss: 5.0359 - accuracy: 0.2642 - val_loss: 2.7707 - val_accuracy: 0.4286 - lr: 0.0010 - 45ms/epoch - 22ms/step
Epoch 5/400
2/2 - 0s - loss: 2.6008 - accuracy: 0.3962 - val_loss: 3.0291 - val_accuracy: 0.3571 - lr: 0.0010 - 35ms/epoch - 17ms/step
Epoch 6/400
2/2 - 0s - loss: 2.4451 - accuracy: 0.3585 - val_loss: 1.7903 - val_accuracy: 0.4286 - lr: 0.0010 - 41ms/epoch - 21ms/step
Epoch 7/400
2/2 - 0s - loss: 1.8704 - accuracy: 0.4340 - val_loss: 1.8123 - val_accuracy: 0.1429 - lr: 0.0010 - 41ms/epoch - 21ms/step
Epoch 8/400
2/2 - 0s - loss: 1

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")

In [ ]:
import matplotlib.pyplot as plt

# Asumiendo que el historial del entrenamiento está guardado en 'history'
# Graficar el descenso de la pérdida (loss)
plt.figure(figsize=(12, 5))

# Gráfico de pérdida (train y validation)
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.title('Descenso de la pérdida durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()

# Gráfico de precisión (train y validation)
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Precisión de entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión de validación')
plt.title('Aumento de la precisión durante el entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()

plt.tight_layout()
plt.show()